In [1]:
from src.MapHandler import Map
from src.Visuals import Visualizer
from src.Drone import Drone
from src import Utils
from src.ParticleFilter import ParticleFilter

import numpy as np

In [2]:
np.random.seed(42)

MapPath = '../Maps/OtherMap.png'

START_POS = np.array([400, 400])
SCAN_SIZE = 50
REAL_DRONE_DISPLACEMENT_VARIANCE = 5
VIRTUAL_DRONE_DISPLACEMENT_VARIANCE = 10
NUM_PARTICLES = 1000
REAL_DRONE_COLOR = (0, 255, 0)
VIRTUAL_DRONE_COLOR = (225, 0, 0)
MEAN_COLOR = (0, 0, 255)
NUM_ITERS = 15
RESAMPLE_VARIANCE = 10
NUM_CHANNEL_BINS = 16
REAL_DRONE_MOVEMENT_RANGE = 25


In [3]:
Map = Map(MapPath)
Visualizer = Visualizer(Map)

# Initialize drones with starting positions as NumPy arrays
RealDrone = Drone(Map, DisplacementVariance=REAL_DRONE_DISPLACEMENT_VARIANCE, ScanSize=SCAN_SIZE, StartPosition=START_POS)
VirtualDrones = [
    Drone(Map, ScanSize=SCAN_SIZE, DisplacementVariance=VIRTUAL_DRONE_DISPLACEMENT_VARIANCE)
    for _ in range(NUM_PARTICLES)
]

ParticleFilter = ParticleFilter(RealDrone, VirtualDrones, RESAMPLE_VARIANCE, NUM_CHANNEL_BINS)


In [4]:
#Visualizer.DisplayImage(Map.GetMapImage(), "Bay")

img_list = []
output_path = "../Output/OtherMapGif.gif"
IMGMapWithDrones = Visualizer.DrawDrones(Map.GetMapImage().copy(), ParticleFilter.VirtualDrones, VIRTUAL_DRONE_COLOR)
IMGMapWithDrones = Visualizer.DrawDrones(IMGMapWithDrones, [ParticleFilter.RealDrone], REAL_DRONE_COLOR)
img_list.append(IMGMapWithDrones)
#Visualizer.DisplayImage(IMGMapWithDrones, "Map with Drones")

for _ in range(NUM_ITERS):

    MovementVector = Utils.GenerateRandomMovementVector(REAL_DRONE_MOVEMENT_RANGE)
    ParticleFilter.RealDrone.Move(MovementVector)
    for D in ParticleFilter.VirtualDrones:
        D.Move(MovementVector)
        
    ParticleFilter.ComputeWeights()
    ParticleFilter.ResampleVirtualDrones()
    ParticleFilter.CullParticles()
    MeanPos = ParticleFilter.EstimatePosition()
    
    IMGMapWithDrones = Visualizer.DrawDrones(Map.GetMapImage().copy(), ParticleFilter.VirtualDrones, VIRTUAL_DRONE_COLOR)
    IMGMapWithDrones = Visualizer.DrawDrones(IMGMapWithDrones, [ParticleFilter.RealDrone], REAL_DRONE_COLOR, radius=5)
    IMGMapWithDrones = Visualizer.DrawDotAtPosition(IMGMapWithDrones, MeanPos, radius=5)
    #Visualizer.DisplayImage(IMGMapWithDrones, "Map with Drones")
    img_list.append(IMGMapWithDrones)

        

In [5]:
Visualizer.compile_images_to_gif(img_list, output_path, duration=1)